# BAIT508_Industry_Analysis
David Chen (30409156)
Wanting Xu (81971392)

## Part 1. Quantitative Analysis of the Industry Sector

In [106]:
# Package Imports and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Load the data of major industry sectors and their corresponding codes
dataFrameMajorGroup = pd.read_csv('data/major_groups.csv')

dataFrame10K = pd.read_csv('data/2020_10K_item1_full.csv')

dataFramePublicFirms = pd.read_csv('data/public_firms.csv')

print(dataFrameMajorGroup.head())
print(dataFramePublicFirms.head())
print(dataFrame10K.head())


   major_group                                        description
0            1                      Agricultural Production Crops
1            2  Agriculture production livestock and animal sp...
2            7                              Agricultural Services
3            8                                           Forestry
4            9                       Fishing hunting and trapping
   gvkey  fyear location      conm     ipodate   sic  prcc_c      ch      ni  \
0   1004   1994      USA  AAR CORP  1988/01/01  5080  13.375  22.487  10.463   
1   1004   1995      USA  AAR CORP  1988/01/01  5080  22.000  33.606  16.012   
2   1004   1996      USA  AAR CORP  1988/01/01  5080  30.250  51.705  23.025   
3   1004   1997      USA  AAR CORP  1988/01/01  5080  38.750  17.222  35.657   
4   1004   1998      USA  AAR CORP  1988/01/01  5080  23.875   8.250  41.671   

     asset     sale       roa  
0  425.814  451.395  0.024572  
1  437.846  504.990  0.036570  
2  529.584  589.328  0.0434

### A. Industry Sector Selection and Data Filtering

#### 1. 
List of major industry sectors and their corresponding codes has been loaded as dataframe `dataFrameMajorGroup`. Among the major industries listed we propose to look into [Industry Name]

In [107]:
selectedMajorGroupCode = 50

#### 2.

In [108]:
# Filter the data in "data/public_firms.csv" to only include the firms belonging to the industry sector(s) you have selected.
def filterByIndustry(dataFrame, majorGroupCode):
    return dataFrame[dataFrame['sic']//100 == majorGroupCode]

print(filterByIndustry(dataFramePublicFirms, selectedMajorGroupCode))

         gvkey  fyear location                         conm     ipodate   sic  \
0         1004   1994      USA                     AAR CORP  1988/01/01  5080   
1         1004   1995      USA                     AAR CORP  1988/01/01  5080   
2         1004   1996      USA                     AAR CORP  1988/01/01  5080   
3         1004   1997      USA                     AAR CORP  1988/01/01  5080   
4         1004   1998      USA                     AAR CORP  1988/01/01  5080   
...        ...    ...      ...                          ...         ...   ...   
203709  187714   2018      USA  WESCO AIRCRAFT HOLDINGS INC  2011/07/28  5080   
203710  187714   2019      USA  WESCO AIRCRAFT HOLDINGS INC  2011/07/28  5080   
204523  197958   2011      CHN        FIFTH SEASON INTL INC         NaN  5051   
204538  198255   2014      USA         WOD RETAIL SOLUTIONS         NaN  5090   
204539  198255   2015      USA         WOD RETAIL SOLUTIONS         NaN  5090   

        prcc_c      ch     

In [ ]:
# 2. How many unique firm-year ("fyear") observations are there in the filtered dataset?

# 3. How many unique firms are there in the filtered dataset?
# 4. How many firms in the filtered dataset have records over all 27 years (1994-2020)?

#### 3.

### B. Preliminary Analysis


In [109]:
#To-Do:

## Part 2. Text Analysis on the Industry Sector

### C. Text Cleaning

In [110]:
#To-Do:

### D. Keyword Analysis

In [111]:
#To-Do:

### E. Word Embedding

In [112]:
#To-Do:

## Part 3. Comprehensive Analysis of One Sample Firm

### F. Firm Analysis and Strategy Suggestion

In [113]:
#To-Do: